# Setup

In [8]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from Levenshtein import ratio as stringSimilarity

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12
fig_size[1] = 5
plt.rcParams["figure.figsize"] = fig_size

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)


In [2]:
tdb = pd.read_pickle('../workproduct-files/cleaned-dataframes/tdb.pkl')
opentdb = pd.read_pickle('../workproduct-files/cleaned-dataframes/opentdb.pkl')
tfyi = pd.read_pickle('../workproduct-files/cleaned-dataframes/tfyi.pkl')
rtg = pd.read_pickle('../workproduct-files/cleaned-dataframes/rtg.pkl')

# Overlap analysis

## String similarity

### Functions and function calls to run 'from Levenshtein import ratio as stringSimilarity' calculations

In [3]:
# calculates similarity between single string y and list of strings in x
def similFunc(x, y):
    ret = []
    ret = x.apply(lambda x: stringSimilarity(x, y))
    ret = ret.tolist()
    return ret

In [4]:
def compareStrings(d, parts):
    
    # 1. input is data (d) Series and nr to split it into (parts)
    # 2. while i <  parts:
    #         run the similFunc on rows / parts and d
    
    for i in range(0, parts):
        
        startInd = int(np.floor(len(d) / parts) * i)
        
        if i == parts - 1:
            endInd = None
        else:
            endInd = int(np.floor(len(d) / parts) * (i + 1))
        
        qSimil = pd.DataFrame(columns = ['q'])
        qSimil['q'] = d[startInd:endInd]
        # qSimil.set_index(part.index)... or something. Saving index prob not necessary. Do if buggy.


        asLists = qSimil['q'].apply(lambda x: similFunc(d, x) )


        calcs = asLists.apply(pd.Series)
        qSimil = pd.concat([qSimil[:], calcs[:]], axis = 1)
        
        fileI = str(i+1)
        if len(fileI) < len(str(parts)):
            pre = '0' * (len(str(parts)) - len(fileI))
            fileI = pre + fileI

        qSimil.to_pickle('../workproduct-files/similarityMatrix/partialDataFrames/similarityM-' + str(fileI) + 'of' + str(parts) + '.pkl')
    

In [5]:
allQuestions = tdb['question'].append([ opentdb['question'], rtg['question'], tfyi['question'] ])
allQuestions.reset_index(drop = True, inplace = True)

In [6]:
similData = allQuestions.head(100)
partsToSplit = 10

%time compareStrings(similData, partsToSplit)

# Here masterID to allQuestions index should be saved for mapping later on.
# Only 'q' Series still passed to functions

Wall time: 267 ms
